In [2]:
from diffusers import StableDiffusionPipeline
import torch
import os

# Check karo ki CUDA available hai ya nahi
device = "cuda" if torch.cuda.is_available() else "cpu"

# Stable Diffusion Pipeline load karo (fp16 only if CUDA is available)
pipe = StableDiffusionPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4",
    torch_dtype=torch.float16 if device == "cuda" else torch.float32
)

# Pipeline ko device pe bhejo
pipe = pipe.to(device)

# Safety checker disable karna ho to ye line use karo (optional)
# pipe.safety_checker = lambda images, **kwargs: (images, False)

# Text prompt
prompt = "ek sundar pahaadon ka drishya, suraj ke neeche"

# Image generate karo
with torch.autocast(device):
    image = pipe(prompt).images[0]

# Output folder bana lo agar nahi hai
os.makedirs("outputs", exist_ok=True)

# Image save karo
image_path = "outputs/pahadon_ka_drisya.png"
image.save(image_path)

print(f"Image generate ho gayi: {image_path}")


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

Image generate ho gayi: outputs/pahadon_ka_drisya.png


In [1]:
pip install langchain openai chromadb unstructured PyMuPDF


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 25.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
pip install -U langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.5 MB/s eta 0:00:00


In [5]:
pip install -U langchain-openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 2.7 MB/s eta 0:00:00


In [6]:
# For PDF loader
from langchain_community.document_loaders import PyMuPDFLoader

# For vector store
from langchain_community.vectorstores import Chroma

# For embeddings
from langchain_openai import OpenAIEmbeddings


In [7]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = PyMuPDFLoader("mypdf.pdf")
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)


In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "sk-tzO0DxUetQXrfBbaa5zR3JW9W8Soae4wPlOSKI30a10A"


In [11]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

loader = PyMuPDFLoader("mypdf.pdf")
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)

embedding = OpenAIEmbeddings()
vectordb = Chroma.from_documents(chunks, embedding)


In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI  # ya FastChat, HuggingFace

# LLM setup (API key already set)
llm = ChatOpenAI(
    model="gpt-3.5-turbo",  # Ya koi aur model
    temperature=0,          # Answer consistent rahe
    openai_api_key="sk-PlOSKI30a10A"  # Optional agar env set kiya ho to
)

# QA chain banayein using retriever from ChromaDB
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectordb.as_retriever(),  # yeh vectordb aapne pehle banaya tha
    return_source_documents=True       # agar aapko source dikhana hai
)

# 🔎 User Query
query = "Is document mein Python aur SQL ka zikr hai kya?"
response = qa_chain.invoke({"query": query})

# 📄 Print Answer
print("Answer:", response["result"])

# 📚 Optional: Show document source
for doc in response["source_documents"]:
    print("---")
    print(doc.page_content[:300])  # pehle 300 characters


/tmp/ipython-input-12-1646908312.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


Answer: Nahi, yeh di gayi context mein Python aur SQL ka zikr nahi hai.
---
AI/ML Fundamentals 
for Product Managers
Darius K, Director of Product Management 
(ServiceNow)
Ex-cofounder of AIR Labs Innovation Unit
Ex-ITSM Intelligence PM & VA Owner
---
/  33  /
Your Continued PM AI/ML Learning 
Journey
Basic AI ML Concepts
•
[Coursera] AI for Everyone
•
DeepLearning.AI Batch AI 
newsletters
•
 Two minute papers 
youtube channel
•
[Stanford] Read the AI 
Index Report
•
Engage in hackathons 
with your team incorporating 
cloud or in-house AI
•
Famil
---
A view on AI 
& innovation
/  3  /
---
require AI
/  7  /
How might you work?
LAUNCH
Production
 Handoff
AI PM Led
Engineering 
Led
AI Product 
Leads
AI Product 
Leads
Platfor
m AI 
Provider
Legal & IT 
Dependenci
es


In [13]:
# 🛠️ Step 1: Install dependencies (agar abhi tak nahi kiya)
# !pip install langchain langchain-community langchain-openai chromadb tiktoken gradio PyPDF2

import gradio as gr
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

# 🧠 Step 2: Define the RAG pipeline
def create_qa_chain(pdf_path):
    loader = PyPDFLoader(pdf_path)
    docs = loader.load()

    splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    splits = splitter.split_documents(docs)

    vectorstore = Chroma.from_documents(splits, OpenAIEmbeddings())
    retriever = vectorstore.as_retriever()

    llm = ChatOpenAI(temperature=0)
    qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)
    return qa_chain

# 🖥️ Step 3: Define Gradio functions
qa_chain = None

def load_pdf(file):
    global qa_chain
    qa_chain = create_qa_chain(file.name)
    return "✅ PDF loaded successfully!"

def ask_question(question):
    if not qa_chain:
        return "⚠️ Pehle PDF upload karo!"
    return qa_chain.run(question)

# 🎛️ Step 4: Create Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("## 🔍 PDF-based RAG Chatbot")

    with gr.Row():
        pdf_file = gr.File(label="Upload PDF")
        load_button = gr.Button("Load PDF")

    status = gr.Textbox(label="Status", interactive=False)
    load_button.click(fn=load_pdf, inputs=[pdf_file], outputs=[status])

    question_input = gr.Textbox(label="Your Question")
    answer_output = gr.Textbox(label="Answer", lines=5)
    ask_btn = gr.Button("Ask")

    ask_btn.click(fn=ask_question, inputs=[question_input], outputs=[answer_output])

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://14dc297a4356e6013b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
